In [2]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.{DataFrame, SparkSession}

import scala.concurrent.duration.DurationInt

object StreamingTest {
  def main(args: Array[String]): Unit = {
    val spark: SparkSession = SparkSession.builder().appName("StreamingTest").master("local[2]").getOrCreate()

    // socket 으로 부터 읽기
    val lines: DataFrame = spark.readStream
      .format("socket")
      .option("host", "localhost")
      .option("port", 12345)
      .load()

    // file 에서 읽기
//    val lines = spark.readStream
//      .format("csv")
//      .option("header", "false")
//      .option("dateFormat", "yyyy-MM-dd")
//      .schema(mySchema)
//      .load("src/main/resources/data")

    val shortLines: DataFrame = lines.filter(length(col("value")) <= 5)
    println(shortLines.isStreaming)

    shortLines.writeStream
      .format("console")
      .outputMode("append")
      .trigger(
        Trigger.ProcessingTime(2.seconds) // every 2 seconds run the query
        // Trigger.Once() // single batch, then terminate
        // Trigger.Continuous(2.seconds) // experimental, every 2 seconds create a batch with whatever you have
      )
      .start()
      .awaitTermination()
  }
}

SyntaxError: invalid syntax (4283772251.py, line 3)